In [3]:
import os

In [4]:
%pwd

'c:\\Users\\vkris\\Text-summarization\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\vkris\\Text-summarization'

In [7]:
from dataclasses import dataclass

In [8]:
from pathlib import  Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [9]:
from src.textsummarizer.constants import *
from src.textsummarizer.utils.common import read_yaml,create_directories

In [52]:
class ConfigurationManager:

    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config



In [53]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from src.textsummarizer.utils.common import get_size

In [54]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config= config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download! with info: \n{headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

   

In [55]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2023-11-25 09:36:42,794: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-25 09:36:42,796: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-25 09:36:42,798: INFO: common: created directory at: artifacts]
[2023-11-25 09:36:42,801: INFO: common: created directory at: artifacts/data_ingestion]
[2023-11-25 09:36:49,001: INFO: 3398498699: artifacts/data_ingestion/data.zip download! with info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9D3C:B8CFB:1AE2E3:2EA0C1:656172D3
Accept-Ranges: bytes
Date: Sat, 25 Nov 2023 04:06:45 GMT
Via: 1.1 varnish
X-Served-By: cache-del21729-DEL
X-Cache: MISS
X-Cache